# Problems So far (August 16)

1.   where to find the genomic reference (The ACGT sequence)
2.   Should I only give the most variable chromosome parts?
3.   The Enformer takes a huge space in memory. I don't think colab could handle it (The torchsummary with gradients would take more than 24 Gbs in RAM)

And the way I should approach the task is still a bit vague.


# Google Colab Setup

In [1]:
! pip install -q kaggle

In [6]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amiralism","key":"cfffa868d2456e4a1cb4a6e389c7af4a"}'}

In [7]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
! kaggle competitions download -c 'open-problems-multimodal'

100% 18.1G/18.1G [14:13<00:00, 25.5MB/s]
100% 18.1G/18.1G [14:13<00:00, 22.7MB/s]


In [9]:
! mkdir data
! unzip open-problems-multimodal.zip -d data

Archive:  open-problems-multimodal.zip
  inflating: data/evaluation_ids.csv  
  inflating: data/metadata.csv       
  inflating: data/metadata_cite_day_2_donor_27678.csv  
  inflating: data/sample_submission.csv  
  inflating: data/test_cite_inputs.h5  
  inflating: data/test_cite_inputs_day_2_donor_27678.h5  
  inflating: data/test_multi_inputs.h5  
  inflating: data/train_cite_inputs.h5  
  inflating: data/train_cite_targets.h5  
  inflating: data/train_multi_inputs.h5  
  inflating: data/train_multi_targets.h5  


In [10]:
! pip install scanpy --quiet
! pip install muon --quiet
! pip install enformer-pytorch --quiet

# Reference

In [ ]:
# ! curl -O https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.6G  100 10.6G    0     0   197M      0  0:00:55  0:00:55 --:--:--  155M


In [ ]:
# ! mkdir reference
# ! tar -xf refdata-gex-GRCh38-2020-A.tar.gz

# Import Libraries

In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns
import re

import gc

import warnings
warnings.filterwarnings('ignore')

import scanpy as sc
import muon

In [12]:
# loading the meta-data for the whole dataset, including CITE-seq data
df_meta = pd.read_csv("/content/data/metadata.csv", index_col='cell_id')
df_meta.head(1)

,day,donor,cell_type,technology
cell_id,,,,
c2150f55becb,2,27678,HSC,citeseq


In [13]:
# selecting only cells from multiome part
df_meta_mul = df_meta[df_meta.technology == "multiome"].copy()
del df_meta
df_meta_mul.head(1)

,day,donor,cell_type,technology
cell_id,,,,
458c2ae2c9b1,2,27678,hidden,multiome


Multiome Dataset (according to kaggle description)

train/test_multi_inputs.h5 - ATAC-seq peak counts transformed with TF-IDF using the default log(TF) * log(IDF) output (chromatin accessibility), with rows corresponding to cells and columns corresponding to the location of the genome whose level of accessibility is measured, here identified by the genomic coordinates on reference genome GRCh38 provided in the 10x References - 2020-A (July 7, 2020).

train_multi_targets.h5 - RNA gene expression levels as library-size normalized and log1p transformed counts for the same cells.

In [26]:
# loading a part of input from training set using pandas (it couldn't be opened using scanpy)
df_multi_train_partial = pd.read_hdf("/content/data/train_multi_inputs.h5", start=0, stop=1000)
df_multi_train_partial.head(3)

gene_id,GL000194.1:114519-115365,GL000194.1:55758-56597,GL000194.1:58217-58957,GL000194.1:59535-60431,GL000195.1:119766-120427,GL000195.1:120736-121603,GL000195.1:137437-138345,GL000195.1:15901-16653,GL000195.1:22357-23209,GL000195.1:23751-24619,...,chrY:7722278-7723128,chrY:7723971-7724880,chrY:7729854-7730772,chrY:7731785-7732664,chrY:7810142-7811040,chrY:7814107-7815018,chrY:7818751-7819626,chrY:7836768-7837671,chrY:7869454-7870371,chrY:7873814-7874709
cell_id,,,,,,,,,,,,,,,,,,,,,
56390cf1b95e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.428336,0.0,0.0,0.0,0.0
fc0c60183c33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9b4a87e22ad0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [15]:
# loading a part of target from training set using pandas
df_multi_tg_partial = pd.read_hdf("/content/data/train_multi_targets.h5", start=0, stop=1000)
df_multi_tg_partial.head(3)

gene_id,ENSG00000121410,ENSG00000268895,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256661,ENSG00000184389,ENSG00000128274,ENSG00000094914,ENSG00000081760,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
cell_id,,,,,,,,,,,,,,,,,,,,,
56390cf1b95e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.893861,0.0,0.0,0.0,0.0,5.583255,0.0,4.893861
fc0c60183c33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
9b4a87e22ad0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.107832,0.0,0.0,0.0,0.0,0.000000,0.0,5.107832


In [16]:
# checking for how sparse the data is
df_multi_train_sum = df_multi_train_partial.sum(axis=0)
df_multi_train_sum

gene_id
GL000194.1:114519-115365    37.848942
GL000194.1:55758-56597      40.180374
GL000194.1:58217-58957      23.534494
GL000194.1:59535-60431      20.312632
GL000195.1:119766-120427    15.214685
                              ...    
chrY:7814107-7815018        35.977760
chrY:7818751-7819626        11.679253
chrY:7836768-7837671        85.008835
chrY:7869454-7870371         0.000000
chrY:7873814-7874709        22.318771
Length: 228942, dtype: float32

In [17]:
# Removing the parts that weren't accessible (around 0.3 of the data remains)
print(np.std(df_multi_train_sum), np.mean(df_multi_train_sum), np.max(df_multi_train_sum))
df_multi_train_sum[df_multi_train_sum > np.std(df_multi_train_sum)]

56.98083 53.56292 368.6152


gene_id
GL000195.1:137437-138345     62.074501
GL000195.1:30343-31248      140.876328
GL000205.2:140323-141187    157.281891
GL000205.2:62941-63796      101.318069
GL000205.2:88697-89483       97.731567
                               ...    
chrY:7085329-7086228         92.750862
chrY:7272239-7272944         88.062790
chrY:7273501-7274337        189.854126
chrY:7282658-7283526         74.084206
chrY:7836768-7837671         85.008835
Length: 69635, dtype: float32

Creating Anndata object to find highly variable features

In [27]:
adata = sc.AnnData(df_multi_train_partial,
    df_multi_train_partial.index.to_frame(),
    df_multi_train_partial.columns.to_frame())
adata

AnnData object with n_obs × n_vars = 1000 × 228942
    obs: 'cell_id'
    var: 'gene_id'

In [28]:
adata.layers['tf-idf'] = adata.X.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=20000)
adata.layers['log-norm'] = adata.X.copy()
atac_hvf = adata[:, adata.var.highly_variable].copy()
atac_hvf

AnnData object with n_obs × n_vars = 1000 × 20000
    obs: 'cell_id'
    var: 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    layers: 'tf-idf', 'log-norm'

In [29]:
# These are the highly variable features
atac_hvf.var["gene_id"]

gene_id
GL000195.1:86114-87015          GL000195.1:86114-87015
KI270711.1:23948-24701          KI270711.1:23948-24701
KI270713.1:36891-37795          KI270713.1:36891-37795
chr10:1000164-1000979            chr10:1000164-1000979
chr10:100324913-100325834    chr10:100324913-100325834
                                       ...            
chrY:20925077-20925995          chrY:20925077-20925995
chrY:21174535-21175450          chrY:21174535-21175450
chrY:2844854-2845729              chrY:2844854-2845729
chrY:7282658-7283526              chrY:7282658-7283526
chrY:7339342-7340266              chrY:7339342-7340266
Name: gene_id, Length: 20000, dtype: object

In [56]:
def extract_genomic_coordinates(serie):
    df = serie.str.split(pat='[-:]', expand=True)
    return df.rename(columns={0:"chromosome", 1:"start", 2:"end"})

In [57]:
extract_genomic_coordinates(atac_hvf.var["gene_id"])

,chromosome,start,end
gene_id,,,
GL000195.1:86114-87015,GL000195.1,86114,87015
KI270711.1:23948-24701,KI270711.1,23948,24701
KI270713.1:36891-37795,KI270713.1,36891,37795
chr10:1000164-1000979,chr10,1000164,1000979
chr10:100324913-100325834,chr10,100324913,100325834
...,...,...,...
chrY:20925077-20925995,chrY,20925077,20925995
chrY:21174535-21175450,chrY,21174535,21175450
chrY:2844854-2845729,chrY,2844854,2845729


# Enformer Pytorch

In [58]:
import torch
from enformer_pytorch import Enformer

In [59]:
model = Enformer.from_hparams(
    dim = 1536,
    depth = 5,
    heads = 4,
    output_heads = dict(human = 1),
    target_length = 896,
)

In [ ]:
# 5 layer, 4 heads
import torchsummary
# with torch.no_grad():
torchsummary.summary(model, input_size=(196_608, 4))

In [ ]:
# # 11 layer
# import torchsummary
# with torch.no_grad():
#     torchsummary.summary(model, input_size=(196_608, 4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Rearrange-1            [-1, 4, 196608]               0
            Conv1d-2          [-1, 768, 196608]          46,848
            Conv1d-3          [-1, 768, 196608]          46,848
       BatchNorm1d-4          [-1, 768, 196608]           1,536
       BatchNorm1d-5          [-1, 768, 196608]           1,536
              GELU-6          [-1, 768, 196608]               0
              GELU-7          [-1, 768, 196608]               0
            Conv1d-8          [-1, 768, 196608]         590,592
            Conv1d-9          [-1, 768, 196608]         590,592
         Residual-10          [-1, 768, 196608]               0
         Residual-11          [-1, 768, 196608]               0
        Rearrange-12        [-1, 768, 98304, 2]               0
        Rearrange-13        [-1, 768, 98304, 2]               0
           Conv2d-14        [-1, 768, 9